# Introduction to PuLP

[PuLP](https://pythonhosted.org/PuLP/) is a python library designed to write Linear Programs (LP) and to transform them in a format compatible with existing LP/ILP solver (such as CBC, GLPK, Gurobi or Cplex).


## Additional resources
Here is a (non-exhaustive) list of resources you can read to help you to formulate your problem or to understand issues:
- [PuLP official documentation](https://www.coin-or.org/PuLP/index.html)
- [Another introduction to PuLP](http://benalexkeen.com/linear-programming-with-python-and-pulp-part-2/)
- [Transform variable multiplication into linear program](https://www.leandro-coelho.com/linearization-product-variables/)

# Simple LP problem : optimizing studying time

To introduce PuLP, a first, very simple problem is proposed.

Let's consider a student who desires having its diploma while working as few as possible.
He has multiple subjects this semester, with different coefficients for the final score.
Also, he has some backgroud and knows how much time he should study each subject in order to have a perfect $20/20$ score.

We will consider that the resulting score at an exam is proportional to the time spent studying.
In addition, no minimum score is required per exam: *the condition to success is simply to have a global score higher than $10/20$ at the whole semester*.

In [1]:
import pulp
import pandas as pd

### Input of the problem

Suppose you have 2 different classes, *Resource Management* and *Theoretical Computer Science (CS)*.
Their coefficients are respectively $2$ and $1$.
As you have a better background in *Theoretical CS*, you consider it will require only 10 hours to have a 20/20 mark and you know that you can't have less than 5/20 even if you spend no time on it before the exam.
For *Resource Management*, you think it requires around 25 hours for the perfect mark, with a minum of 3/20.

An average of 10/20 is needed to have your semester.
However, as you can over/under perform slightly the day of the exam, you prefer to target 12/20.

| Subject            | Coefficient | Time for having 20/20   | Minimum Grade |
|:-------------------|-------------|-------------------------|---------------|
| Resource Management| 2           | 20 hours                |   3/20        |
| Theorical CS       | 1           | 10 hours                |   5/20        |

The inputs of the problems are simply *constant values* (constant for a given *instance* of the problem).
To avoid [Magic Values](https://en.wikipedia.org/wiki/Magic_number_(programming)) in the code, we can simply define them as python values:


In [2]:
# Coefficients
coef_rm = 2
coef_tcs = 1

# Time for perfect score
#maxtime_rm = 25
#maxtime_tcs = 10
maxtime_rm = 20
maxtime_tcs = 10

mingrade_rm = 3/20
mingrade_tcs = 4/20

# the targetted semester score (as a fraction in [0, 1])
target = 12/20

### Formulation as a Linear Program

We want to *minimize* the total time spent studying.
Let's note $t_{rm}$ the time spent for Resource Management, and $t_{tcs}$ the time spent for Theorical Computer Science.

$$
\begin{equation}
\text{Minimize: } t_{rm} + t_{tcs}
\end{equation}
$$

Multiple constraints should be set:
- $t_{rm}$ and $t_{tcs}$ are positive numbers
- $t_{rm}$ and $t_{tcs}$ can't be more than the time required for having the perfect score
- the score expected for each must, considering the coefficients, give at least the targetted score to the whole semester

$$
\begin{align}
\text{Subject to:} \quad & t_{rm} \geq 0 \\
& t_{tcs} \geq 0 \\
& t_{rm} \leq \textit{maxtime}_{rm} \\
& t_{tcs} \leq \textit{maxtime}_{tcs} \\
& \frac{\left(\textit{min}_{tcs} +  \frac{t_{tcs} \cdot (20/20 - \textit{min}_{tcs})}{\textit{maxtime}_{tcs}} \right) \cdot \textit{coef}_{tcs}  
           + \left(\textit{min}_{rm} +  \frac{t_{rm} \cdot (20/20 - \textit{min}_{rm})}{\textit{maxtime}_{rm}} \right) \cdot \textit{coef}_{rm}}
       {\textit{coef}_{rm} + \textit{coef}_{tcs}} \geq \textit{target}
\end{align}
$$

**Note:** as $\textit{min}_{xxx}$, $\textit{coef}_{xxx}$ and $\textit{maxtime}_{xxx}$ are inputs of the problem, the last constraint **is** linear (possible to rewrite as: $t_{tcs} \cdot a + t_{rm} \cdot b - c \geq 0$).


#### Decision variable declaration for PuLP

In [3]:
prob = pulp.LpProblem("Optimize the time spent to obtain your semester")

/home/chuxclub/.local/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [4]:
time_rm = pulp.LpVariable("Time_ResourceManagement")
time_tcs = pulp.LpVariable("Time_TheoricalCS")

#### Adding the constraints to the problem

In [5]:
# time spent must be positive
prob += time_rm >= 0
prob += time_tcs >= 0

In [6]:
# no more than 20/20 to each class, spending more time is useless
prob += time_rm <= maxtime_rm
prob += time_tcs <= maxtime_tcs

In [7]:
# at least 12/20 ('target') for the weighted average (semester mark)

# expressions for simplifications of the equations
score_rm = mingrade_rm + time_rm * (1 / maxtime_rm) * (20/20 - mingrade_rm)
score_tcs = mingrade_tcs + time_tcs * (1 / maxtime_tcs) * (20/20 - mingrade_tcs)

# real constraint to add
prob += (score_rm * coef_rm + score_tcs * coef_tcs) * (1 / (coef_rm + coef_tcs)) >= target


#### Objective of the problem

**Note:** Usually, the *sense* and *objective* is defined at the beginning of the problem in PuLP (see the doc).

In [8]:
prob.sense = pulp.LpMinimize
prob.setObjective(time_rm + time_tcs)

## Resolution of this instance

If the solver used by PuLP is able to find a solution to the problem, it will return a status `Optimal` and it will be possible to access to the value of all the descision variables and constraints declared previously.

In [9]:
prob.solve()
pulp.LpStatus[prob.status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/chuxclub/.local/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/d691f64663c34b80b7b740b8eae4333f-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/d691f64663c34b80b7b740b8eae4333f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 19 RHS
At line 25 BOUNDS
At line 28 ENDATA
Problem MODEL has 5 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-4) rows, 2 (0) columns and 2 (-4) elements
0  Obj 5.7823526 Primal inf 9.511764 (1)
1  Obj 15.294118
Optimal - objective value 15.294118
After Postsolve, objective 15.294118, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 15.29411765 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallcl

'Optimal'

It is possible to access to all the variables of the problem and their final values.

In [10]:
for v in prob.variables():
    print(v.name, '=', v.varValue)

Time_ResourceManagement = 15.294118
Time_TheoricalCS = 0.0


Another way to get the value of a variable is by the `LpVariable` instance directly, through the method `value()`.

In [11]:
import datetime
time_spent = datetime.timedelta(hours=time_rm.value())
print('I need to work on Resource Management for exactly {}'.format(str(time_spent)))

I need to work on Resource Management for exactly 15:17:38.824800


It must be noted that the value of an expression (`LpAffineExpression`) can be evaluated the same way.

In [12]:
print('Will result in score of {:.2f}/20 in RM and {:.2f}/20 in TCS.'.format(score_rm.value() * 20, score_tcs.value() * 20))

Will result in score of 16.00/20 in RM and 4.00/20 in TCS.


## Use of indexed variables

The same problem may be writen differently if we want to support any number of subjects ($N$).

$$
\begin{equation}
\text{Minimize: } \sum_{i=1}^N{t_{i}}
\end{equation}
$$

$$
\text{Subject to:} \\
\begin{align}
\forall i \in [1, N] \quad & t_{i} \geq 0 \\
& t_{i} \leq \textit{maxtime}_{i} \\
\end{align}\\
\sum_{i=1}^N{
             \frac{
                    \left(\textit{min}_{i} +  \frac{t_{i} \cdot (20/20 - \textit{min}_{i})}{\textit{maxtime}_{i}} \right) \cdot \textit{coef}_{i}
                  }
                  {\sum_{j=1}^N{ \textit{coef}_{j} }}
             } \geq \textit{target}
$$


In [13]:
N = 2
subjects = ['Resource Management', 'Theorical CS']

# Coefficients
coefs = [coef_rm, coef_tcs]

# Time for perfect score
maxtimes = [maxtime_rm, maxtime_tcs]
mingrades = [mingrade_rm, mingrade_tcs]

target = 12/20

In [14]:
prob_any = pulp.LpProblem("Optimize the time spent to obtain any semester with any number of subjects")

In [15]:
times = [pulp.LpVariable('Time_' + name) for name in subjects]

#### Adding the constraints to the problem

In [16]:
for i in range(N):
    prob_any += times[i] >= 0
    prob_any += times[i] <= maxtimes[i]
    
# expressions for simplifications
scores = []
for i in range(N):
    scores.append(mingrades[i] + times[i] * (1 / maxtimes[i]) * (20/20 - mingrades[i]))

*Warning*: do **not** use the python builtin `sum()` for building a Pulp expression.
Instead, use `pulp.lpSum()`.

However, you should use python's `sum()` if you are only summing constants.

In [17]:
prob_any += pulp.lpSum(scores[i] * coefs[i] * (1 / sum(coefs)) for i in range(N)) >= target

#### Objective of the problem and resolution

In [18]:
prob_any.sense = pulp.LpMinimize
prob_any.setObjective(pulp.lpSum(times))

In [19]:
prob_any.solve()
pulp.LpStatus[prob_any.status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/chuxclub/.local/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/03643463b5d04f57a1ea2e8ba4502166-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/03643463b5d04f57a1ea2e8ba4502166-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 19 RHS
At line 25 BOUNDS
At line 28 ENDATA
Problem MODEL has 5 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-4) rows, 2 (0) columns and 2 (-4) elements
0  Obj 5.7823526 Primal inf 9.511764 (1)
1  Obj 15.294118
Optimal - objective value 15.294118
After Postsolve, objective 15.294118, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 15.29411765 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallcl

'Optimal'

In [20]:
for v in prob_any.variables():
    print(v.name, '=', v.varValue)

Time_Resource_Management = 15.294118
Time_Theorical_CS = 0.0
